# Create Grid Visualizations from the subset order

This tutorial shows how to create grid visualization from the [Global Subset Tool](https://modis.ornl.gov/globalsubset/) order. 

Let's first import the required Python modules.

In [1]:
import requests
import tarfile
import glob
from os import path
import xarray as xr
import matplotlib.pyplot as plt
import hvplot.xarray
import xyzservices.providers as xyz
import cartopy.crs as ccrs 
from datetime import datetime

### Place Global Subsets Order
Let's submit [Global Subset requests](https://modis.ornl.gov/globalsubset/) for MODIS/Terra Gross Primary Productivity Gap-Filled product ([MOD17A2HGF](https://doi.org/10.5067/MODIS/MOD17A2HGF.061)) for a location (Latitude: `35.9625N`, Longitude: `-84.295822W`) and time period `2018-04-01` to `2018-07-31`. We will request a subset area 3km around the location (area of about 6.5km x 6.5km). 

It will take some time to finish processing the subset orders, and once the subsets are ready, you will receive an email with the link to the order delivery pages like the following: https://modis.ornl.gov/subsetdata/03Jul2025_07:54:54_085407192L35.9625L-84.295822S13L13_MOD17A2HGF_1

### Download Files
Let's download the GeoTIFF files from the order delivery page and save it to the folder here. The [`GTiff.tar.gz`](https://modis.ornl.gov/subsetdata/03Jul2025_07:54:54_085407192L35.9625L-84.295822S13L13_MOD17A2HGF_1/tif/GTiff.tar.gz) file contains the compressed version of the GeoTIFF files.

In [2]:
def download_tar(orderid):
    """
    function to download and untar file file
    """
    order_url = 'https://modis.ornl.gov/subsetdata' # order URL
    geotiff_tar = 'GTiff.tar.gz' # tar file name
    
    # get requests
    r = requests.get(f'{order_url}/{orderid}/tif/{geotiff_tar}', stream=True)
    r.raise_for_status()
    
    # download tar file
    with open(f'output/{geotiff_tar}', 'wb') as f:
        f.write(r.raw.read())
    
    # extract tar file
    with tarfile.open(f'output/{geotiff_tar}') as f:
        f.extractall('output') 

# order id from the Global Subsets Tool
# REPLACE with your own ORDER ID
orderid = '03Jul2025_07:54:54_085407192L35.9625L-84.295822S13L13_MOD17A2HGF_1'

# download and uncompress GeoTiffs
download_tar(orderid)

Now that the files are downloaded to the `output` folder let's print all the GeoTIFF subset files.

In [3]:
subset_f = sorted(glob.glob("output/MOD17A2HGF*"))
subset_f

['output/MOD17A2HGF.A2018097.h11v05.061.2021362123028_Gpp_500m.tif',
 'output/MOD17A2HGF.A2018097.h11v05.061.2021362123028_PsnNet_500m.tif',
 'output/MOD17A2HGF.A2018097.h11v05.061.2021362123028_Psn_QC_500m.tif',
 'output/MOD17A2HGF.A2018105.h11v05.061.2021362133256_Gpp_500m.tif',
 'output/MOD17A2HGF.A2018105.h11v05.061.2021362133256_PsnNet_500m.tif',
 'output/MOD17A2HGF.A2018105.h11v05.061.2021362133256_Psn_QC_500m.tif',
 'output/MOD17A2HGF.A2018113.h11v05.061.2021362153207_Gpp_500m.tif',
 'output/MOD17A2HGF.A2018113.h11v05.061.2021362153207_PsnNet_500m.tif',
 'output/MOD17A2HGF.A2018113.h11v05.061.2021362153207_Psn_QC_500m.tif',
 'output/MOD17A2HGF.A2018121.h11v05.061.2021362164413_Gpp_500m.tif',
 'output/MOD17A2HGF.A2018121.h11v05.061.2021362164413_PsnNet_500m.tif',
 'output/MOD17A2HGF.A2018121.h11v05.061.2021362164413_Psn_QC_500m.tif',
 'output/MOD17A2HGF.A2018129.h11v05.061.2021362174415_Gpp_500m.tif',
 'output/MOD17A2HGF.A2018129.h11v05.061.2021362174415_PsnNet_500m.tif',
 'outpu

There are one geotiff file for each date and band combination. There are two science bands `Gpp_500m` and `PsnNet_500m` as well as a quality control band `Psn_QC_500m`. Let's open all `Gpp` bands using `xarray` and plot.

In [4]:
def get_file_date(f):
    """returns date from the geotiff file path"""
    date_str = path.basename(f).split(".")[1][1:]
    return datetime.strptime(date_str, "%Y%j")
    
# selecting gpp files
gpp_f = sorted(glob.glob("output/MOD17A2HGF*Gpp_500m.tif"))
# Concatenate along band dimension
gpp = xr.concat([xr.open_dataset(x, engine="rasterio") for x in gpp_f], dim='band')
# rename band to time
gpp = gpp.rename({"band": "time"})
# assign actual values to time
gpp["time"] = [get_file_date(x) for x in gpp_f]
# plot
gpp.rio.reproject(ccrs.GOOGLE_MERCATOR).hvplot.quadmesh(
    cmap='viridis', tiles=xyz.Esri.WorldImagery).opts(
    title = 'Gpp_500m band')

:DynamicMap   [time]
   :Overlay
      .Tiles.I    :Tiles   [x,y]
      .QuadMesh.I :QuadMesh   [x,y]   (band_data)

## Mask fill values and apply scale factor
As we see above, few pixels have a value 32766, which is a fill value of the datasets. The following 7 fill values are defined for non-vegetated pixels.

```
32767 = _Fillvalue
32766 = land cover assigned as perennial salt or Water bodies
32765 = land cover assigned as barren,sparse veg (rock,tundra,desert)
32764 = land cover assigned as perennial snow,ice.
32763 = land cover assigned as "permanent" wetlands/inundated marshland
32762 = land cover assigned as urban/built-up
32761 = land cover assigned as "unclassified" or (not able to determine)
```
Also, `Gpp_500m` and `PsnNet_500m` bands of MOD17A2GH dataset have a scale factor of `0.0001`.

Please refer to the [MOD17 user guide](https://lpdaac.usgs.gov/documents/972/MOD17_User_Guide_V61.pdf) for more information on fill values and scale_factor. 

In [5]:
# mask the fill values and apply scale factor
scale_factor = 0.0001
gpp_masked = gpp.where((gpp < 32761) | (gpp >32767)) * scale_factor
gpp_masked.rio.reproject(ccrs.GOOGLE_MERCATOR).hvplot.quadmesh(
    cmap='viridis', tiles=xyz.Esri.WorldImagery).opts(
    title = 'Scaled Gpp_500m band (Fill Values Masked)')

:DynamicMap   [time]
   :Overlay
      .Tiles.I    :Tiles   [x,y]
      .QuadMesh.I :QuadMesh   [x,y]   (band_data)

## Apply Quality Filter
The `Psn_QC_500m` band provides the QC information. Please refer to the [MOD17 user guide](https://lpdaac.usgs.gov/documents/972/MOD17_User_Guide_V61.pdf) for more information on quality filter. 

Let's read all the `Psn_QC_500m` bands from the subset order GeoTiffs and plot.

In [6]:
# selecting gpp files
psn_qc_f = sorted(glob.glob("output/MOD17A2HGF*Psn_QC_500m.tif"))
# Concatenate along band dimension
psn_qc = xr.concat([xr.open_dataset(x, engine="rasterio") for x in psn_qc_f], dim='band')
# rename band to time
psn_qc = psn_qc.rename({"band": "time"})
# assign actual values to time
psn_qc["time"] = [get_file_date(x) for x in psn_qc_f]
psn_qc.rio.reproject(ccrs.GOOGLE_MERCATOR).hvplot.quadmesh(
    cmap='viridis', tiles=xyz.Esri.WorldImagery).opts(
    title = 'Psn_QC Quality Band')

:DynamicMap   [time]
   :Overlay
      .Tiles.I    :Tiles   [x,y]
      .QuadMesh.I :QuadMesh   [x,y]   (band_data)

As we see above the QC bands are 8-bit integers with values ranging from 0 to 255. There are different bits defined for MOD17A2H:
```
BITS BITFIELD
-------------

0,0 MODLAND_QC bits
 '0' = Good Quality (main algorithm with or without saturation)
 '1' = Other Quality (back-up algorithm or fill values)

1,1 SENSOR
 '0' = Terra
 '1' = Aqua

2,2 DEADDETECTOR
 '0' = Detectors apparently fine for up to 50% of channels 1,2
 '1' = Dead detectors caused >50% adjacent detector retrieval

3,4 CLOUDSTATE (this inherited from Aggregate_QC bits {0,1} cloud state)
 '00' = 0 Significant clouds NOT present (clear)
 '01' = 1 Significant clouds WERE present
 '10' = 2 Mixed cloud present on pixel
 '11' = 3 Cloud state not defined, assumed clear

5,7 SCF_QC (3-bit, (range '000'..100') 5 level Confidence Quality score.
 '000' = 0, Main (RT) method used, best result possible (no saturation)
 '001' = 1, Main (RT) method used with saturation. Good,very usable
 '010' = 2, Main (RT) method failed due to bad geometry, empirical algorithm used
 '011' = 3, Main (RT) method failed due to problems other than geometry, empirical algorithm used
 '100' = 4, Pixel not produced at all, value couldn't be retrieved (possible reasons: bad L1Bdata, unusable MOD09GA data)
```

For simplicity, we can only keep the pixels where `SFC_QC` is `001`, which indicates "good and very usable pixels".